In [1]:
#bibliotecas necessárias
import requests
import pandas as pd

#Realizando as requisições
users = requests.get('https://fakestoreapi.com/users')
products = requests.get('https://fakestoreapi.com/products')
carts = requests.get('https://fakestoreapi.com/carts')
cat = requests.get('https://fakestoreapi.com/products/categories')

# Dados de *Usuários*

In [2]:
data = users.json()
usuarioDF = pd.json_normalize(data)

#remover colunas desnecessárias
usuarioDF.drop(['username','__v', 'password','address.geolocation.lat', 'address.geolocation.long'], axis='columns', inplace=True)
#renomear colunas
usuarioDF.rename({'id':'userId','phone':'Telefone','address.city':'Cidade', 'address.street': 'Rua', 'address.number':'Numero', 'address.zipcode': 'CEP', 'name.firstname':'Nome', 'name.lastname': 'Sobrenome'},  axis=1, inplace=True)
#usuarioDF

# Dados de Produtos

In [3]:
data = products.json()

productsDF = pd.json_normalize(data)
#remover colunas desnecessárias
productsDF.drop(['image','rating.rate', 'rating.count'], axis='columns', inplace=True)

#renomear colunas
productsDF.rename({'id':'productId','title':'Titulo', 'price':'Preço', 'description':'Descricao', 'category':'Categoria'}, axis='columns', inplace=True)
#productsDF

# Carrinho

In [4]:
data = carts.json()

cartsDF = pd.json_normalize(data)
cartsDF.drop(['__v'], axis='columns', inplace=True)
#cartsDF

###juntando tabelas

In [7]:
joinCartsDF = usuarioDF.join(
     cartsDF.set_index(["userId"]),
     on=["userId"],
     how="inner")

###ordenando por datas

In [8]:
joinCartsDF.sort_values(by=['date'], ascending=False)
joinCartsDF

,userId,email,Telefone,Cidade,Rua,Numero,CEP,Nome,Sobrenome,id,date,products
0,1,john@gmail.com,1-570-236-7033,kilcoole,new road,7682,12926-3874,john,doe,1,2020-03-02T00:00:00.000Z,"[{'productId': 1, 'quantity': 4}, {'productId'..."
0,1,john@gmail.com,1-570-236-7033,kilcoole,new road,7682,12926-3874,john,doe,2,2020-01-02T00:00:00.000Z,"[{'productId': 2, 'quantity': 4}, {'productId'..."
1,2,morrison@gmail.com,1-570-236-7033,kilcoole,Lovers Ln,7267,12926-3874,david,morrison,3,2020-03-01T00:00:00.000Z,"[{'productId': 1, 'quantity': 2}, {'productId'..."
2,3,kevin@gmail.com,1-567-094-1345,Cullman,Frances Ct,86,29567-1452,kevin,ryan,4,2020-01-01T00:00:00.000Z,"[{'productId': 1, 'quantity': 4}]"
2,3,kevin@gmail.com,1-567-094-1345,Cullman,Frances Ct,86,29567-1452,kevin,ryan,5,2020-03-01T00:00:00.000Z,"[{'productId': 7, 'quantity': 1}, {'productId'..."
3,4,don@gmail.com,1-765-789-6734,San Antonio,Hunters Creek Dr,6454,98234-1734,don,romer,6,2020-03-01T00:00:00.000Z,"[{'productId': 10, 'quantity': 2}, {'productId..."
7,8,william@gmail.com,1-478-001-0890,mesa,vally view ln,1342,96378-0245,william,hopkins,7,2020-03-01T00:00:00.000Z,"[{'productId': 18, 'quantity': 1}]"


###Salvando arquivo json


In [9]:
joinCartsDF.to_json('temp.json', orient='records', lines=True)